In [40]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import TomekLinks

In [2]:
numerical = pd.read_csv('numerical.csv',index_col=0)
categorical = pd.read_csv('categorical.csv')
y = pd.read_csv('Y.csv')

In [3]:
df= pd.DataFrame(numerical.isna().sum()/len(numerical)).reset_index()
df.columns = ['column_name', 'nulls']
df[df['nulls']>0].sort_values(by='nulls',ascending=False)

,column_name,nulls


In [4]:
df= pd.DataFrame(categorical.isna().sum()/len(categorical)).reset_index()
df.columns = ['column_name', 'nulls']
df[df['nulls']>0].sort_values(by='nulls',ascending=False)

,column_name,nulls
6,SOLIH,0.935019


In [5]:
categorical.drop(['SOLIH'], axis=1,inplace=True)

In [6]:
df= pd.DataFrame(categorical.isna().sum()/len(categorical)).reset_index()
df.columns = ['column_name', 'nulls']
df[df['nulls']>0].sort_values(by='nulls',ascending=False)

,column_name,nulls


In [7]:
categorical.drop(['Unnamed: 0'], axis=1,inplace=True)
categorical.drop(['Unnamed: 0.1'], axis=1,inplace=True)

In [8]:
categorical

,STATE,ZIP,CLUSTER,HOMEOWNR,RFA_2,RFA_2R,GEOCODE2,DOMAIN_A,DOMAIN_B
0,IL,61081,36,U,L4E,L,C,T,2
1,CA,91326,14,H,L2G,L,A,S,1
2,NC,27017,43,U,L4E,L,C,R,2
3,CA,95953,44,U,L4E,L,C,R,2
4,FL,33176,16,H,L2F,L,A,S,2
...,...,...,...,...,...,...,...,...,...
95407,OTHER,99504,27,U,L1G,L,C,C,2
95408,TX,77379,24,H,L1F,L,A,C,1
95409,MI,48910,30,U,L3E,L,B,C,3
95410,CA,91320,24,H,L4F,L,A,C,1


In [9]:
categorical.dtypes

STATE       object
ZIP         object
CLUSTER      int64
HOMEOWNR    object
RFA_2       object
RFA_2R      object
GEOCODE2    object
DOMAIN_A    object
DOMAIN_B     int64
dtype: object

In [10]:
categorical['CLUSTER'] = categorical['CLUSTER'].astype(object)
categorical['DOMAIN_B'] = categorical['DOMAIN_B'].astype(object)

In [11]:
numerical.dtypes

Unnamed: 0      int64
ODATEDW         int64
TCODE           int64
DOB             int64
AGE           float64
               ...   
AVGGIFT       float64
CONTROLN        int64
HPHONE_D        int64
RFA_2F          int64
CLUSTER2      float64
Length: 322, dtype: object

In [12]:
categorical

,STATE,ZIP,CLUSTER,HOMEOWNR,RFA_2,RFA_2R,GEOCODE2,DOMAIN_A,DOMAIN_B
0,IL,61081,36,U,L4E,L,C,T,2
1,CA,91326,14,H,L2G,L,A,S,1
2,NC,27017,43,U,L4E,L,C,R,2
3,CA,95953,44,U,L4E,L,C,R,2
4,FL,33176,16,H,L2F,L,A,S,2
...,...,...,...,...,...,...,...,...,...
95407,OTHER,99504,27,U,L1G,L,C,C,2
95408,TX,77379,24,H,L1F,L,A,C,1
95409,MI,48910,30,U,L3E,L,B,C,3
95410,CA,91320,24,H,L4F,L,A,C,1


In [13]:
numerical

,Unnamed: 0,ODATEDW,TCODE,DOB,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,...,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
0,0,8901,0,3712,60.000000,0.0,5.345699,0,0,39,...,9402,10.0,9512,8911,4.000000,7.741935,95515,0,4,39.0
1,1,9401,1,5202,46.000000,6.0,9.000000,16,0,15,...,9512,25.0,9512,9310,18.000000,15.666667,148535,0,2,1.0
2,2,9001,1,0,61.611649,3.0,1.000000,2,0,20,...,9207,5.0,9512,9001,12.000000,7.481481,15078,1,4,60.0
3,3,8701,0,2801,70.000000,1.0,4.000000,2,0,23,...,9411,10.0,9512,8702,9.000000,6.812500,172556,1,4,41.0
4,4,8601,0,2001,78.000000,3.0,2.000000,60,1,28,...,9601,15.0,9601,7903,14.000000,6.864865,7112,1,2,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,95407,9601,1,0,61.611649,0.0,5.345699,0,14,36,...,9602,25.0,9602,9602,8.093739,25.000000,184568,0,1,12.0
95408,95408,9601,1,5001,48.000000,7.0,9.000000,1,0,31,...,9603,20.0,9603,9603,8.093739,20.000000,122706,1,1,2.0
95409,95409,9501,1,3801,60.000000,0.0,5.345699,0,0,18,...,9501,10.0,9610,9410,3.000000,8.285714,189641,1,3,34.0
95410,95410,8601,0,4005,58.000000,7.0,5.345699,0,0,28,...,9608,18.0,9701,8612,4.000000,12.146341,4693,1,4,11.0


In [14]:
numerical.drop(['Unnamed: 0'], axis=1,inplace=True)

In [15]:
scaler = MinMaxScaler()

for col in numerical.columns:
    numerical[col] = scaler.fit_transform(numerical[[col]])

numerical.head()

,ODATEDW,TCODE,DOB,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,...,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
0,0.426523,0.000000,0.382286,0.608247,0.000000,0.593967,0.000000,0.000000,0.393939,0.343434,...,0.863139,0.010,0.045226,0.927939,0.003676,0.006465,0.498045,0.0,1.000000,0.622951
1,0.784946,0.000014,0.535736,0.463918,0.857143,1.000000,0.066390,0.000000,0.151515,0.555556,...,0.913321,0.025,0.045226,0.969489,0.016544,0.014399,0.774510,0.0,0.333333,0.000000
2,0.498208,0.000014,0.000000,0.624862,0.428571,0.111111,0.008299,0.000000,0.202020,0.292929,...,0.774179,0.005,0.045226,0.937311,0.011029,0.006204,0.078617,1.0,1.000000,0.967213
3,0.283154,0.000000,0.288465,0.711340,0.142857,0.444444,0.008299,0.000000,0.232323,0.141414,...,0.867245,0.010,0.045226,0.906175,0.008272,0.005534,0.899764,1.0,1.000000,0.655738
4,0.211470,0.000000,0.206076,0.793814,0.428571,0.222222,0.248963,0.010101,0.282828,0.090909,...,0.953923,0.015,0.492462,0.822972,0.012868,0.005586,0.037079,1.0,0.333333,0.409836


In [16]:
categorical.nunique()

STATE          12
ZIP         19938
CLUSTER        53
HOMEOWNR        2
RFA_2          14
RFA_2R          1
GEOCODE2        5
DOMAIN_A        5
DOMAIN_B        4
dtype: int64

In [17]:
categorical.drop(['ZIP'], axis=1,inplace=True)

In [18]:
one_hot_names = []
for col in categorical.columns:
    col_uniques = sorted(categorical[col].astype(str).unique())
    for unique in col_uniques:
        one_hot_names.append(col+"_"+unique)
        
categorical = pd.DataFrame(OneHotEncoder().fit_transform(categorical.astype(str)).toarray())
categorical.columns = one_hot_names
categorical.head()

,STATE_CA,STATE_FL,STATE_GA,STATE_IL,STATE_IN,STATE_MI,STATE_MO,STATE_NC,STATE_OTHER,STATE_TX,...,GEOCODE2_U,DOMAIN_A_C,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U,DOMAIN_B_1,DOMAIN_B_2,DOMAIN_B_3,DOMAIN_B_4
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [19]:
X = pd.concat([numerical, categorical], axis=1)

In [20]:
X

,ODATEDW,TCODE,DOB,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,...,GEOCODE2_U,DOMAIN_A_C,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U,DOMAIN_B_1,DOMAIN_B_2,DOMAIN_B_3,DOMAIN_B_4
0,0.426523,0.000000,0.382286,0.608247,0.000000,0.593967,0.000000,0.000000,0.393939,0.343434,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,0.784946,0.000014,0.535736,0.463918,0.857143,1.000000,0.066390,0.000000,0.151515,0.555556,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.498208,0.000014,0.000000,0.624862,0.428571,0.111111,0.008299,0.000000,0.202020,0.292929,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.283154,0.000000,0.288465,0.711340,0.142857,0.444444,0.008299,0.000000,0.232323,0.141414,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.211470,0.000000,0.206076,0.793814,0.428571,0.222222,0.248963,0.010101,0.282828,0.090909,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,0.928315,0.000014,0.000000,0.624862,0.000000,0.593967,0.000000,0.141414,0.363636,0.474747,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
95408,0.928315,0.000014,0.515036,0.484536,1.000000,1.000000,0.004149,0.000000,0.313131,0.434343,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
95409,0.856631,0.000014,0.391452,0.608247,0.000000,0.593967,0.000000,0.000000,0.181818,0.464646,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
95410,0.211470,0.000000,0.412461,0.587629,1.000000,0.593967,0.000000,0.000000,0.282828,0.353535,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [22]:
y

,Unnamed: 0,TARGET_B,TARGET_D
0,0,0,0.0
1,1,0,0.0
2,2,0,0.0
3,3,0,0.0
4,4,0,0.0
...,...,...,...
95407,95407,0,0.0
95408,95408,0,0.0
95409,95409,0,0.0
95410,95410,1,18.0


In [23]:
y.drop(['Unnamed: 0'], axis=1,inplace=True)

In [24]:
y.drop(['TARGET_D'], axis=1, inplace=True)

In [25]:
y

,TARGET_B
0,0
1,0
2,0
3,0
4,0
...,...
95407,0
95408,0
95409,0
95410,1


In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.head()

,ODATEDW,TCODE,DOB,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,...,GEOCODE2_U,DOMAIN_A_C,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U,DOMAIN_B_1,DOMAIN_B_2,DOMAIN_B_3,DOMAIN_B_4
85225,0.928315,0.000389,0.000000,0.624862,0.000000,0.593967,0.000000,0.0,0.292929,0.363636,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
70004,0.498208,0.000000,0.350257,0.649485,0.571429,0.593967,0.000000,0.0,0.232323,0.232323,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
88133,0.354839,0.000014,0.360556,0.639175,1.000000,0.666667,0.008299,0.0,0.252525,0.272727,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
79106,0.354839,0.000028,0.267868,0.731959,0.285714,0.593967,0.000000,0.0,0.343434,0.191919,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
35476,0.426523,0.000000,0.000000,0.624862,0.142857,0.593967,0.000000,0.0,0.373737,0.464646,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [29]:
print(y_train.shape)

(76329, 1)


In [30]:
print(X_train.shape)

(76329, 417)


In [33]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [34]:
accuracy = model.score(X_test, y_test)
accuracy

0.9487501965099827

In [35]:
y.value_counts()

TARGET_B
0           90569
1            4843
dtype: int64

In [36]:
smote = SMOTE()

x_resampled,y_resampled=smote.fit_resample(X,y)
y_resampled.value_counts()

TARGET_B
0           90569
1           90569
dtype: int64

In [37]:
X_train, X_test, y_train, y_test = train_test_split(x_resampled, y_resampled, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.6025725957822679

In [38]:
RUS=RandomUnderSampler(random_state=0)
x_resampled,y_resampled=RUS.fit_resample(X,y)

y_resampled.value_counts()

TARGET_B
0           4843
1           4843
dtype: int64

In [39]:
X_train, X_test, y_train, y_test = train_test_split(x_resampled, y_resampled, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.5810113519091847

In [41]:
tl = TomekLinks(sampling_strategy='majority')
X_tl, y_tl=tl.fit_resample(X,y)
y_tl.value_counts()

TARGET_B
0           88668
1            4843
dtype: int64

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X_tl, y_tl, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.947013848045768